In [ ]:
# libraries
import pandas as pd
import numpy as np
import os
import billboard
import re
import ast
from dotenv import load_dotenv
from lyricsgenius import Genius

In [ ]:
# load in environment variables
load_dotenv()
genius_token = os.getenv('GENIUS_TOKEN')

In [64]:
# writing a regex function for breaking up the artist format to match the Kaggle database
# input: 
    # artist: string
# returns
    # artist_list: a list of strings containing the artist(s)

def artist_regex(artist_str):
    artist_list = re.split(r'\s*(Featuring|&|,)\s*', artist_str)
    artist_list = [artist.strip() for artist in artist_list if artist.strip() not in ['Featuring', 'ft', '&', 'and' ',']]

    if len(artist_list) == 1: # solo artist case
        return artist_list[0]
    
    return artist_list
    

In [65]:
# call for the billboard 100s from 2010-2023
chart_data_list = []
years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

for y in years:
    chart = billboard.ChartData('hot-100-songs', year=y)

    for song in chart:
        chart_data_list.append({
            'name': song.title,
            'artists': artist_regex(song.artist),
            'Performer': song.artist,
            'year': int(y)
        })


chart_data = pd.DataFrame(chart_data_list)
chart_data['artists'] = chart_data['artists'].apply(str)
chart_data.head()

,name,artists,Performer,year
0,TiK ToK,Ke$ha,Ke$ha,2010
1,Need You Now,Lady Antebellum,Lady Antebellum,2010
2,"Hey, Soul Sister",Train,Train,2010
3,California Gurls,"['Katy Perry', 'Snoop Dogg']",Katy Perry Featuring Snoop Dogg,2010
4,OMG,"['Usher', 'will.i.am']",Usher Featuring will.i.am,2010


In [66]:
chart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1398 entries, 0 to 1397
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       1398 non-null   object
 1   artists    1398 non-null   object
 2   Performer  1398 non-null   object
 3   year       1398 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 43.8+ KB


In [67]:
chart_data = chart_data.replace("Ke$ha", "Kesha")
chart_data

,name,artists,Performer,year
0,TiK ToK,Kesha,Kesha,2010
1,Need You Now,Lady Antebellum,Lady Antebellum,2010
2,"Hey, Soul Sister",Train,Train,2010
3,California Gurls,"['Katy Perry', 'Snoop Dogg']",Katy Perry Featuring Snoop Dogg,2010
4,OMG,"['Usher', 'will.i.am']",Usher Featuring will.i.am,2010
...,...,...,...,...
1393,"Bzrp Music Sessions, Vol. 53","['Bizarrap', 'Shakira']",Bizarrap & Shakira,2023
1394,Meltdown,"['Travis Scott', 'Drake']",Travis Scott Featuring Drake,2023
1395,Put It On Da Floor Again,"['Latto', 'Cardi B']",Latto Featuring Cardi B,2023
1396,Bloody Mary,Lady Gaga,Lady Gaga,2023


In [68]:
# since the kaggle database has the artists in a string which is a list of strings, we need to convert it

def list_eval(str):
    try:
        return ast.literal_eval(str)
    except:
        return str


In [69]:
# pull kaggle datasets
df_kaggle_1 = pd.read_csv('kaggle_data/songs_with_attributes_and_lyrics.csv')
df_kaggle_2 = pd.read_csv('kaggle_data/Hot 100 Audio Features.csv')
df_kaggle_3 = pd.read_csv('kaggle_data/billboard_24years_lyrics_spotify.csv')

In [70]:
# merge the dataframes, left on the chart data
merged_df = chart_data.merge(df_kaggle_1, on=['name', 'artists'], how = 'left')

# drop some unneeded columns
merged_df = merged_df.drop(columns = ['id', 'album_name'])

merged_df

,name,artists,Performer,year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,lyrics
0,TiK ToK,Kesha,Kesha,2010,0.755,0.837,2,-2.718,0,0.1420,0.0991,0.000000,0.289,0.714,120.028,199693.0,Wake up in the morning feelin' like P. Diddy (...
1,TiK ToK,Kesha,Kesha,2010,0.755,0.832,2,-2.741,0,0.1160,0.0746,0.000001,0.291,0.735,120.032,199693.0,Wake up in the morning feelin' like P. Diddy (...
2,TiK ToK,Kesha,Kesha,2010,0.737,0.839,2,-2.028,0,0.1700,0.1030,0.000000,0.319,0.718,120.042,200187.0,Wake up in the morning feelin' like P. Diddy (...
3,Need You Now,Lady Antebellum,Lady Antebellum,2010,0.588,0.724,4,-4.379,1,0.0315,0.0348,0.000095,0.239,0.377,107.986,236787.0,"Picture perfect memories, scattered all around..."
4,Need You Now,Lady Antebellum,Lady Antebellum,2010,0.581,0.717,4,-4.433,1,0.0318,0.0298,0.000186,0.243,0.316,107.884,236440.0,"Picture perfect memories, scattered all around..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077,Meltdown,"['Travis Scott', 'Drake']",Travis Scott Featuring Drake,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2078,Put It On Da Floor Again,"['Latto', 'Cardi B']",Latto Featuring Cardi B,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2079,Bloody Mary,Lady Gaga,Lady Gaga,2023,0.584,0.638,9,-6.437,0,0.0302,0.0134,0.000003,0.101,0.456,99.970,244760.0,Money\n ♪\n Oh\n Love is just a history that t...
2080,Bloody Mary,Lady Gaga,Lady Gaga,2023,0.584,0.638,9,-6.437,0,0.0302,0.0134,0.000003,0.101,0.456,99.970,244760.0,Money\n ♪\n Oh\n Love is just a history that t...


In [71]:
# drop some unnneded columns before merging
df_kaggle_2 = df_kaggle_2.drop(columns=['index', 'SongID', 'spotify_track_id', 'spotify_track_preview_url', 
                          'spotify_track_explicit','spotify_track_album', 'time_signature', 
                          'spotify_track_popularity'])
df_kaggle_2

,Performer,Song,spotify_genre,spotify_track_duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Bill Black's Combo,-twistin'-White Silver Sands,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Augie Rios,¿Dònde Està Santa Claus? (Where Is Santa Claus?),['novelty'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Andy Williams,......And Roses And Roses,"['adult standards', 'brill building pop', 'eas...",166106.0,0.154,0.185,5.0,-14.063,1.0,0.0315,0.91100,0.000267,0.1120,0.150,83.969
3,Sandy Nelson,...And Then There Were Drums,"['rock-and-roll', 'space age pop', 'surf music']",172066.0,0.588,0.672,11.0,-17.278,0.0,0.0361,0.00256,0.745000,0.1450,0.801,121.962
4,Britney Spears,...Baby One More Time,"['dance pop', 'pop', 'post-teen pop']",211066.0,0.759,0.699,0.0,-5.745,0.0,0.0307,0.20200,0.000131,0.4430,0.907,92.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29498,Lil Tjay Featuring Fivio Foreign & Pop Smoke,Zoo York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29499,Future,Zoom,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",278429.0,0.852,0.438,9.0,-7.673,1.0,0.4260,0.01450,0.000000,0.2630,0.627,150.945
29500,Lil' Boosie Featuring Yung Joc,Zoom,"['baton rouge rap', 'deep southern trap']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29501,Herb Alpert & The Tijuana Brass,Zorba The Greek,"['adult standards', 'easy listening', 'lounge']",264853.0,0.531,0.642,5.0,-12.702,1.0,0.3230,0.15400,0.279000,0.0584,0.192,82.107


In [72]:
# do another merge
merged_df_2 = merged_df.merge(df_kaggle_2, left_on=['name', 'Performer'], right_on= ['Song', 'Performer'], 
                how = 'left', suffixes=['_df1', '_df2'])
merged_df_2

,name,artists,Performer,year,danceability_df1,energy_df1,key_df1,loudness_df1,mode_df1,speechiness_df1,...,energy_df2,key_df2,loudness_df2,mode_df2,speechiness_df2,acousticness_df2,instrumentalness_df2,liveness_df2,valence_df2,tempo_df2
0,TiK ToK,Kesha,Kesha,2010,0.755,0.837,2,-2.718,0,0.1420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TiK ToK,Kesha,Kesha,2010,0.755,0.832,2,-2.741,0,0.1160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TiK ToK,Kesha,Kesha,2010,0.737,0.839,2,-2.028,0,0.1700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Need You Now,Lady Antebellum,Lady Antebellum,2010,0.588,0.724,4,-4.379,1,0.0315,...,0.622,4.0,-5.535,1.0,0.0303,0.0927,0.000636,0.2,0.231,107.943
4,Need You Now,Lady Antebellum,Lady Antebellum,2010,0.581,0.717,4,-4.433,1,0.0318,...,0.622,4.0,-5.535,1.0,0.0303,0.0927,0.000636,0.2,0.231,107.943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,Meltdown,"['Travis Scott', 'Drake']",Travis Scott Featuring Drake,2023,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2298,Put It On Da Floor Again,"['Latto', 'Cardi B']",Latto Featuring Cardi B,2023,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2299,Bloody Mary,Lady Gaga,Lady Gaga,2023,0.584,0.638,9,-6.437,0,0.0302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2300,Bloody Mary,Lady Gaga,Lady Gaga,2023,0.584,0.638,9,-6.437,0,0.0302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
# combine audio feature columns with combine first
features = [
    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'
]

# loop through the featyres
for feature in features:
    merged_df_2[feature] = merged_df_2[f'{feature}_df1'].combine_first(merged_df_2[f'{feature}_df2'])

# some additional columns that don't follow the pattern
merged_df_2['duration_ms_new'] = merged_df_2['duration_ms'].combine_first(merged_df_2['spotify_track_duration_ms'])

merged_df_2.drop(columns = ['danceability_df1',
       'energy_df1', 'key_df1', 'loudness_df1', 'mode_df1', 'speechiness_df1',
       'acousticness_df1', 'instrumentalness_df1', 'liveness_df1',
       'valence_df1', 'tempo_df1', 'duration_ms', 'spotify_track_duration_ms', 'danceability_df2',
       'energy_df2', 'key_df2', 'loudness_df2', 'mode_df2', 'speechiness_df2',
       'acousticness_df2', 'instrumentalness_df2', 'liveness_df2',
       'valence_df2', 'tempo_df2', 'Song'], inplace = True)

merged_df_2

,name,artists,Performer,year,lyrics,spotify_genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms_new
0,TiK ToK,Kesha,Kesha,2010,Wake up in the morning feelin' like P. Diddy (...,NaN,0.755,0.837,2,-2.718,0,0.1420,0.0991,0.000000,0.289,0.714,120.028,199693.0
1,TiK ToK,Kesha,Kesha,2010,Wake up in the morning feelin' like P. Diddy (...,NaN,0.755,0.832,2,-2.741,0,0.1160,0.0746,0.000001,0.291,0.735,120.032,199693.0
2,TiK ToK,Kesha,Kesha,2010,Wake up in the morning feelin' like P. Diddy (...,NaN,0.737,0.839,2,-2.028,0,0.1700,0.1030,0.000000,0.319,0.718,120.042,200187.0
3,Need You Now,Lady Antebellum,Lady Antebellum,2010,"Picture perfect memories, scattered all around...","['contemporary country', 'country', 'country d...",0.588,0.724,4,-4.379,1,0.0315,0.0348,0.000095,0.239,0.377,107.986,236787.0
4,Need You Now,Lady Antebellum,Lady Antebellum,2010,"Picture perfect memories, scattered all around...","['contemporary country', 'country', 'country d...",0.581,0.717,4,-4.433,1,0.0318,0.0298,0.000186,0.243,0.316,107.884,236440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,Meltdown,"['Travis Scott', 'Drake']",Travis Scott Featuring Drake,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2298,Put It On Da Floor Again,"['Latto', 'Cardi B']",Latto Featuring Cardi B,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2299,Bloody Mary,Lady Gaga,Lady Gaga,2023,Money\n ♪\n Oh\n Love is just a history that t...,NaN,0.584,0.638,9,-6.437,0,0.0302,0.0134,0.000003,0.101,0.456,99.970,244760.0
2300,Bloody Mary,Lady Gaga,Lady Gaga,2023,Money\n ♪\n Oh\n Love is just a history that t...,NaN,0.584,0.638,9,-6.437,0,0.0302,0.0134,0.000003,0.101,0.456,99.970,244760.0


In [74]:
# drop unneeded columns
df_kaggle_3.drop(columns = ['ranking', 'songurl', 'titletext', 'url', 
                            'uri', 'type', 'id', 'track_href', 'analysis_url',
                            'time_signature'], inplace = True)
df_kaggle_3

,song,band_singer,year,lyrics,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,Breathe,Faith Hill,2000,I can feel the magic floating in the air\nBein...,0.529,0.496,7.0,-9.007,1.0,0.0290,0.17300,0.000000,0.2510,0.278,136.859,250547.0
1,Smooth,Santana,2000,"Man, it's a hot one\nLike seven inches from th...",0.609,0.923,9.0,-3.908,1.0,0.0338,0.16000,0.000005,0.2950,0.961,115.996,294987.0
2,Smooth,Rob Thomas,2000,"Man, it's a hot one\nLike seven inches from th...",0.590,0.637,9.0,-9.171,1.0,0.0301,0.00225,0.807000,0.2990,0.724,115.983,244924.0
3,Maria Maria,Santana,2000,"Ladies and gents, turn up your sound systems\n...",0.777,0.601,2.0,-5.931,1.0,0.1260,0.04060,0.002010,0.0348,0.680,97.911,261973.0
4,Maria Maria,The Product G&B,2000,Turn up this sound system\nTo the sound of Car...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3392,Watermelon Moonshine,Lainey Wilson,2023,It was right after senior year\nJust before th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3393,Cheerleader,OMI,2015,When I need motivation\nMy one solution is my ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3394,Felix Jaehn,OMI,2015,When I need motivation\nMy one solution is my ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3395,Waves,Mr Probz,2015,My face above the water\nMy feet can't touch t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
# do another merge
merged_df_3 = merged_df_2.merge(df_kaggle_3, left_on=['name', 'year'], right_on= ['song', 'year'], 
                how = 'left', suffixes=['_df2', '_df3'])
merged_df_3

,name,artists,Performer,year,lyrics_df2,spotify_genre,danceability_df2,energy_df2,key_df2,loudness_df2,...,key_df3,loudness_df3,mode_df3,speechiness_df3,acousticness_df3,instrumentalness_df3,liveness_df3,valence_df3,tempo_df3,duration_ms
0,TiK ToK,Kesha,Kesha,2010,Wake up in the morning feelin' like P. Diddy (...,NaN,0.755,0.837,2,-2.718,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TiK ToK,Kesha,Kesha,2010,Wake up in the morning feelin' like P. Diddy (...,NaN,0.755,0.832,2,-2.741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TiK ToK,Kesha,Kesha,2010,Wake up in the morning feelin' like P. Diddy (...,NaN,0.737,0.839,2,-2.028,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Need You Now,Lady Antebellum,Lady Antebellum,2010,"Picture perfect memories, scattered all around...","['contemporary country', 'country', 'country d...",0.588,0.724,4,-4.379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Need You Now,Lady Antebellum,Lady Antebellum,2010,"Picture perfect memories, scattered all around...","['contemporary country', 'country', 'country d...",0.581,0.717,4,-4.433,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2865,Meltdown,"['Travis Scott', 'Drake']",Travis Scott Featuring Drake,2023,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2866,Put It On Da Floor Again,"['Latto', 'Cardi B']",Latto Featuring Cardi B,2023,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2867,Bloody Mary,Lady Gaga,Lady Gaga,2023,Money\n ♪\n Oh\n Love is just a history that t...,NaN,0.584,0.638,9,-6.437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2868,Bloody Mary,Lady Gaga,Lady Gaga,2023,Money\n ♪\n Oh\n Love is just a history that t...,NaN,0.584,0.638,9,-6.437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
# combine audio feature columns with combine first
features = [
    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
    'lyrics'
]

# loop through the featyres
for feature in features:
    merged_df_3[feature] = merged_df_3[f'{feature}_df2'].combine_first(merged_df_3[f'{feature}_df3'])

# some additional columns that don't follow the pattern
merged_df_3['duration_ms'] = merged_df_3['duration_ms_new'].combine_first(merged_df_3['duration_ms'])

merged_df_3.drop(columns = ['danceability_df2',
       'energy_df2', 'key_df2', 'loudness_df2', 'mode_df2', 'speechiness_df2',
       'acousticness_df2', 'instrumentalness_df2', 'liveness_df2',
       'valence_df2', 'tempo_df2', 'duration_ms_new', 'danceability_df3',
       'energy_df3', 'key_df3', 'loudness_df3', 'mode_df3', 'speechiness_df3',
       'acousticness_df3', 'instrumentalness_df3', 'liveness_df3',
       'valence_df3', 'tempo_df3', 'lyrics_df2', 'lyrics_df3', 'song', 'band_singer'], inplace = True)

merged_df_3

,name,artists,Performer,year,spotify_genre,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics
0,TiK ToK,Kesha,Kesha,2010,NaN,199693.0,0.755,0.837,2,-2.718,0,0.1420,0.0991,0.000000,0.289,0.714,120.028,Wake up in the morning feelin' like P. Diddy (...
1,TiK ToK,Kesha,Kesha,2010,NaN,199693.0,0.755,0.832,2,-2.741,0,0.1160,0.0746,0.000001,0.291,0.735,120.032,Wake up in the morning feelin' like P. Diddy (...
2,TiK ToK,Kesha,Kesha,2010,NaN,200187.0,0.737,0.839,2,-2.028,0,0.1700,0.1030,0.000000,0.319,0.718,120.042,Wake up in the morning feelin' like P. Diddy (...
3,Need You Now,Lady Antebellum,Lady Antebellum,2010,"['contemporary country', 'country', 'country d...",236787.0,0.588,0.724,4,-4.379,1,0.0315,0.0348,0.000095,0.239,0.377,107.986,"Picture perfect memories, scattered all around..."
4,Need You Now,Lady Antebellum,Lady Antebellum,2010,"['contemporary country', 'country', 'country d...",236440.0,0.581,0.717,4,-4.433,1,0.0318,0.0298,0.000186,0.243,0.316,107.884,"Picture perfect memories, scattered all around..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2865,Meltdown,"['Travis Scott', 'Drake']",Travis Scott Featuring Drake,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\nYeah\nTensions is definitely rising\nT'd up ...
2866,Put It On Da Floor Again,"['Latto', 'Cardi B']",Latto Featuring Cardi B,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2867,Bloody Mary,Lady Gaga,Lady Gaga,2023,NaN,244760.0,0.584,0.638,9,-6.437,0,0.0302,0.0134,0.000003,0.101,0.456,99.970,Money\n ♪\n Oh\n Love is just a history that t...
2868,Bloody Mary,Lady Gaga,Lady Gaga,2023,NaN,244760.0,0.584,0.638,9,-6.437,0,0.0302,0.0134,0.000003,0.101,0.456,99.970,Money\n ♪\n Oh\n Love is just a history that t...


In [160]:
# drop duplicates
df_unique= merged_df_3.drop_duplicates(subset=['name', 'artists', 'year'])

# drop the bonus performer column
df_unique = df_unique.drop(columns =['Performer'])

# drop NaN for audio features, use danceability as the main representation 
# usually if it's missing one, it's missing all features
df_unique.dropna(subset=['danceability'], inplace= True)

df_unique

,name,artists,year,spotify_genre,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics
0,TiK ToK,Kesha,2010,NaN,199693.0,0.755,0.837,2,-2.718,0,0.1420,0.09910,0.000000,0.2890,0.714,120.028,Wake up in the morning feelin' like P. Diddy (...
3,Need You Now,Lady Antebellum,2010,"['contemporary country', 'country', 'country d...",236787.0,0.588,0.724,4,-4.379,1,0.0315,0.03480,0.000095,0.2390,0.377,107.986,"Picture perfect memories, scattered all around..."
5,"Hey, Soul Sister",Train,2010,"['neo mellow', 'pop', 'pop rock']",216667.0,0.675,0.885,1,-4.432,0,0.0436,0.21700,0.000000,0.0860,0.768,97.030,"Hey, hey\n Hey-hey, hey-hey\n Hey-hey, hey-hey..."
7,California Gurls,"['Katy Perry', 'Snoop Dogg']",2010,NaN,234653.0,0.788,0.753,0,-3.710,1,0.0536,0.00380,0.000000,0.1800,0.403,125.005,Greetings loved ones\n Let's take a journey\n ...
11,OMG,"['Usher', 'will.i.am']",2010,"['atl hip hop', 'dance pop', 'pop', 'pop rap',...",269493.0,0.781,0.745,4.0,-5.810,0.0,0.0332,0.19800,0.000011,0.3600,0.326,129.998,"Oh my gosh\nBaby, let me\nI did it again, so I..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845,Last Christmas,Wham!,2023,['new wave pop'],265613.0,0.748,0.692,11,-5.976,0,0.0271,0.33400,0.000000,0.2890,0.875,106.612,"Ah, ah-ah, ooh-whoa\n Ah-ah\n Last Christmas, ..."
2849,Handle On You,Parker McCollum,2023,NaN,219333.0,0.588,0.931,6,-3.925,1,0.0318,0.00411,0.000000,0.3450,0.625,105.027,I went and bought the biggest bottle they got ...
2861,I Wrote The Book,Morgan Wallen,2023,NaN,181767.0,0.692,0.808,2,-3.757,1,0.0396,0.08340,0.000037,0.0913,0.802,143.977,When it comes to\n Hitchin' the boat up\n Back...
2867,Bloody Mary,Lady Gaga,2023,NaN,244760.0,0.584,0.638,9,-6.437,0,0.0302,0.01340,0.000003,0.1010,0.456,99.970,Money\n ♪\n Oh\n Love is just a history that t...


In [161]:
# convert the artists and genre strings back into lists
df_unique['artists'] = df_unique['artists'].apply(list_eval)
df_unique['spotify_genre'] = df_unique['spotify_genre'].apply(list_eval)

In [162]:
# get the subset of songs missing lyrics
missing_lyrics = df_unique[df_unique['lyrics'].isnull()]
missing_lyrics

,name,artists,year,spotify_genre,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics
15,Love The Way You Lie,"[Eminem, Rihanna]",2010,"[detroit hip hop, g funk, hip hop, rap]",263373.0,0.749,0.925,10.0,-5.034,1.0,0.2270,0.2410,0.000000,0.5200,0.6410,86.989,NaN
28,Nothin' On You,"[B.o.B, Bruno Mars]",2010,"[atl hip hop, dance pop, hip hop, pop, pop rap...",268320.0,0.688,0.853,10.0,-5.814,1.0,0.0493,0.3860,0.000000,0.0862,0.7430,103.993,NaN
45,DJ Got Us Fallin' In Love,"[Usher, Pitbull]",2010,"[atl hip hop, dance pop, pop, pop rap, r&b, ur...",220800.0,0.663,0.861,7.0,-3.398,0.0,0.1090,0.0338,0.000000,0.0820,0.6540,119.963,NaN
50,Sexy Chick,"[David Guetta, Akon]",2010,"[dance pop, edm, pop]",194346.0,0.809,0.562,11.0,-7.694,0.0,0.0534,0.0816,0.000418,0.1050,0.7980,130.003,NaN
64,Just A Dream,Nelly,2010,"[dance pop, hip hop, pop, pop rap, rap, southe...",237800.0,0.531,0.752,1.0,-6.161,1.0,0.0305,0.0421,0.000000,0.1200,0.1030,89.917,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135,You Broke Me First.,Tate McRae,2021,"[alt z, dance pop, electropop, pop, post-teen ...",169265.0,0.667,0.373,4.0,-9.389,1.0,0.0500,0.7850,0.000000,0.0906,0.0823,124.148,NaN
2156,Telepatia,Kali Uchis,2021,"[colombian pop, pop]",160191.0,0.653,0.524,11.0,-9.016,0.0,0.0502,0.1120,0.000000,0.2030,0.5530,83.970,NaN
2226,pov,Ariana Grande,2021,"[dance pop, pop, post-teen pop]",201882.0,0.487,0.534,0.0,-5.664,0.0,0.0555,0.3600,0.000000,0.1000,0.1730,131.798,NaN
2319,Things A Man Oughta Know,Lainey Wilson,2021,"[contemporary country, country pop]",203373.0,0.659,0.683,3.0,-5.623,1.0,0.0312,0.5130,0.000005,0.1330,0.3970,139.931,NaN


In [99]:
# functions to help with pulling the artist to search for lyrics

def pull_artist(artist):
    if isinstance(artist, list):
        artist_str = artist[0]
    else:
        artist_str = artist
    
    return artist_str

def search_lyrics(track, artist, genius):
    try:
        artist_str = pull_artist(artist)
        song = genius.search_song(track, artist_str)
        lyrics = song.lyrics

    except:
        return None

    return lyrics


In [100]:
# set up for loop to search the lyrics
genius = Genius(genius_token, timeout= 10)

tracks = missing_lyrics['name'].tolist()
artists = missing_lyrics['artists'].tolist()
new_lyrics =[]

for track, artist in zip(tracks, artists):
    lyrics = search_lyrics(track, artist, genius)
    new_lyrics.append(
        {
            'name': track,
            'artists': artist,
            'lyrics':lyrics
        }
    )

new_lyrics_df = pd.DataFrame(new_lyrics)
new_lyrics_df

Searching for "Love The Way You Lie" by Eminem...
Done.
Searching for "Nothin' On You" by B.o.B...
Done.
Searching for "DJ Got Us Fallin' In Love" by Usher...
Done.
Searching for "Sexy Chick" by David Guetta...
Done.
Searching for "Just A Dream" by Nelly...
Done.
Searching for "Like A G6" by Far*East Movement...
Done.
Searching for "Whataya Want From Me" by Adam Lambert...
Done.
Searching for "All The Right Moves" by OneRepublic...
Done.
Searching for "According To You" by Orianthi...
Done.
Searching for "Today Was A Fairytale" by Taylor Swift...
Done.
Searching for "King Of Anything" by Sara Bareilles...
Done.
Searching for "Teach Me How To Dougie" by Cali Swag District...
Done.
Searching for "Try Sleeping With A Broken Heart" by Alicia Keys...
Done.
Searching for "Moves Like Jagger" by Maroon 5...
Done.
Searching for "Black And Yellow" by Wiz Khalifa...
Done.
Searching for "Hey Baby (Drop It To The Floor)" by Pitbull...
Done.
Searching for "I'm On One" by DJ Khaled...
Done.
Searching

,name,artists,lyrics
0,Love The Way You Lie,"[Eminem, Rihanna]",364 ContributorsTranslationsTürkçeEspañolPortu...
1,Nothin' On You,"[B.o.B, Bruno Mars]",77 ContributorsTranslationsPortuguêsNothin’ On...
2,DJ Got Us Fallin' In Love,"[Usher, Pitbull]",76 ContributorsDJ Got Us Fallin’ in Love Lyric...
3,Sexy Chick,"[David Guetta, Akon]",22 ContributorsSexy Chick Lyrics[Verse: Akon]\...
4,Just A Dream,Nelly,74 ContributorsJust a Dream Lyrics[Chorus]\nUh...
...,...,...,...
136,You Broke Me First.,Tate McRae,94 ContributorsTranslationsDanskTürkçeEspañolP...
137,Telepatia,Kali Uchis,86 ContributorsTranslationsPortuguês日本語Deutsch...
138,pov,Ariana Grande,192 ContributorsTranslationsPolskiTürkçeEspaño...
139,Things A Man Oughta Know,Lainey Wilson,9 ContributorsThings a Man Oughta Know Lyrics[...


In [184]:
new_lyrics_df = pd.DataFrame(new_lyrics)
new_lyrics_df

,name,artists,lyrics
0,Love The Way You Lie,"[Eminem, Rihanna]",364 ContributorsTranslationsTürkçeEspañolPortu...
1,Nothin' On You,"[B.o.B, Bruno Mars]",77 ContributorsTranslationsPortuguêsNothin’ On...
2,DJ Got Us Fallin' In Love,"[Usher, Pitbull]",76 ContributorsDJ Got Us Fallin’ in Love Lyric...
3,Sexy Chick,"[David Guetta, Akon]",22 ContributorsSexy Chick Lyrics[Verse: Akon]\...
4,Just A Dream,Nelly,74 ContributorsJust a Dream Lyrics[Chorus]\nUh...
...,...,...,...
136,You Broke Me First.,Tate McRae,94 ContributorsTranslationsDanskTürkçeEspañolP...
137,Telepatia,Kali Uchis,86 ContributorsTranslationsPortuguês日本語Deutsch...
138,pov,Ariana Grande,192 ContributorsTranslationsPolskiTürkçeEspaño...
139,Things A Man Oughta Know,Lainey Wilson,9 ContributorsThings a Man Oughta Know Lyrics[...


In [183]:
# do a quick clean to remove all the beginning bits of the text

def clean_lyric_beginning(lyric_str):
    if lyric_str == None:
        return None
    else:
        lyric = lyric_str.split("Lyrics", 1)
        clean_lyric = re.sub(r'^\[[^\]]+\]\s*', '', lyric[1].strip())
        return clean_lyric

In [185]:
# do some initial cleaning
new_lyrics_df['lyrics'] = new_lyrics_df['lyrics'].apply(clean_lyric_beginning)
new_lyrics_df

,name,artists,lyrics
0,Love The Way You Lie,"[Eminem, Rihanna]",Just gonna stand there and watch me burn?\nWel...
1,Nothin' On You,"[B.o.B, Bruno Mars]",Beautiful girls all over the world\nI could be...
2,DJ Got Us Fallin' In Love,"[Usher, Pitbull]","USHER, USHER, USHER\nYeah, man\n\n[Verse 1: US..."
3,Sexy Chick,"[David Guetta, Akon]","Yes, I can see her\n'Cause every girl here wan..."
4,Just A Dream,Nelly,"Uh, uh, uh (Hey)\nI was thinking 'bout her (Uh..."
...,...,...,...
136,You Broke Me First.,Tate McRae,"You broke me first, ah\n\n[Verse 1]\nMaybe you..."
137,Telepatia,Kali Uchis,[Coro]\nQuién lo diría que se podría\nHacer el...
138,pov,Ariana Grande,It's like you got superpowers\nTurn my minutes...
139,Things A Man Oughta Know,Lainey Wilson,I can hook a trailer on a two-inch hitch\nI ca...


In [186]:
# merge the new lyrics back in
df_final = df_unique.merge(new_lyrics_df, on= 'name', how = 'left', suffixes=['_df1', '_df2'])
df_final

,name,artists_df1,year,spotify_genre,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics_df1,artists_df2,lyrics_df2
0,TiK ToK,Kesha,2010,NaN,199693.0,0.755,0.837,2,-2.718,0,0.1420,0.09910,0.000000,0.2890,0.714,120.028,Wake up in the morning feelin' like P. Diddy (...,NaN,NaN
1,Need You Now,Lady Antebellum,2010,"[contemporary country, country, country dawn, ...",236787.0,0.588,0.724,4,-4.379,1,0.0315,0.03480,0.000095,0.2390,0.377,107.986,"Picture perfect memories, scattered all around...",NaN,NaN
2,"Hey, Soul Sister",Train,2010,"[neo mellow, pop, pop rock]",216667.0,0.675,0.885,1,-4.432,0,0.0436,0.21700,0.000000,0.0860,0.768,97.030,"Hey, hey\n Hey-hey, hey-hey\n Hey-hey, hey-hey...",NaN,NaN
3,California Gurls,"[Katy Perry, Snoop Dogg]",2010,NaN,234653.0,0.788,0.753,0,-3.710,1,0.0536,0.00380,0.000000,0.1800,0.403,125.005,Greetings loved ones\n Let's take a journey\n ...,NaN,NaN
4,OMG,"[Usher, will.i.am]",2010,"[atl hip hop, dance pop, pop, pop rap, r&b, ur...",269493.0,0.781,0.745,4.0,-5.810,0.0,0.0332,0.19800,0.000011,0.3600,0.326,129.998,"Oh my gosh\nBaby, let me\nI did it again, so I...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,Last Christmas,Wham!,2023,[new wave pop],265613.0,0.748,0.692,11,-5.976,0,0.0271,0.33400,0.000000,0.2890,0.875,106.612,"Ah, ah-ah, ooh-whoa\n Ah-ah\n Last Christmas, ...",NaN,NaN
1148,Handle On You,Parker McCollum,2023,NaN,219333.0,0.588,0.931,6,-3.925,1,0.0318,0.00411,0.000000,0.3450,0.625,105.027,I went and bought the biggest bottle they got ...,NaN,NaN
1149,I Wrote The Book,Morgan Wallen,2023,NaN,181767.0,0.692,0.808,2,-3.757,1,0.0396,0.08340,0.000037,0.0913,0.802,143.977,When it comes to\n Hitchin' the boat up\n Back...,NaN,NaN
1150,Bloody Mary,Lady Gaga,2023,NaN,244760.0,0.584,0.638,9,-6.437,0,0.0302,0.01340,0.000003,0.1010,0.456,99.970,Money\n ♪\n Oh\n Love is just a history that t...,NaN,NaN


In [187]:
# do a combine first
df_final['lyrics'] = df_final['lyrics_df1'].combine_first(df_final['lyrics_df2'])

# drop unneeded columns/rename the artists
df_final.drop(columns = ['lyrics_df1', 'lyrics_df2', 'artists_df2'], inplace = True)
df_final.rename(columns={'artists_df1': 'artists'}, inplace= True)

# do a duplicates drop check
df_final.drop_duplicates(subset=['name', 'year'], inplace= True)
df_final

,name,artists,year,spotify_genre,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics
0,TiK ToK,Kesha,2010,NaN,199693.0,0.755,0.837,2,-2.718,0,0.1420,0.09910,0.000000,0.2890,0.714,120.028,Wake up in the morning feelin' like P. Diddy (...
1,Need You Now,Lady Antebellum,2010,"[contemporary country, country, country dawn, ...",236787.0,0.588,0.724,4,-4.379,1,0.0315,0.03480,0.000095,0.2390,0.377,107.986,"Picture perfect memories, scattered all around..."
2,"Hey, Soul Sister",Train,2010,"[neo mellow, pop, pop rock]",216667.0,0.675,0.885,1,-4.432,0,0.0436,0.21700,0.000000,0.0860,0.768,97.030,"Hey, hey\n Hey-hey, hey-hey\n Hey-hey, hey-hey..."
3,California Gurls,"[Katy Perry, Snoop Dogg]",2010,NaN,234653.0,0.788,0.753,0,-3.710,1,0.0536,0.00380,0.000000,0.1800,0.403,125.005,Greetings loved ones\n Let's take a journey\n ...
4,OMG,"[Usher, will.i.am]",2010,"[atl hip hop, dance pop, pop, pop rap, r&b, ur...",269493.0,0.781,0.745,4.0,-5.810,0.0,0.0332,0.19800,0.000011,0.3600,0.326,129.998,"Oh my gosh\nBaby, let me\nI did it again, so I..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,Last Christmas,Wham!,2023,[new wave pop],265613.0,0.748,0.692,11,-5.976,0,0.0271,0.33400,0.000000,0.2890,0.875,106.612,"Ah, ah-ah, ooh-whoa\n Ah-ah\n Last Christmas, ..."
1148,Handle On You,Parker McCollum,2023,NaN,219333.0,0.588,0.931,6,-3.925,1,0.0318,0.00411,0.000000,0.3450,0.625,105.027,I went and bought the biggest bottle they got ...
1149,I Wrote The Book,Morgan Wallen,2023,NaN,181767.0,0.692,0.808,2,-3.757,1,0.0396,0.08340,0.000037,0.0913,0.802,143.977,When it comes to\n Hitchin' the boat up\n Back...
1150,Bloody Mary,Lady Gaga,2023,NaN,244760.0,0.584,0.638,9,-6.437,0,0.0302,0.01340,0.000003,0.1010,0.456,99.970,Money\n ♪\n Oh\n Love is just a history that t...


In [165]:
# write out the audio features to csv
df_final_audio_features = df_final.drop(columns = ['lyrics'])
df_final_audio_features.to_csv('audio_features.csv')

In [188]:
# eliminate any None in lyrics
df_final_lyrics = df_final.dropna(subset=['lyrics'])
df_final_lyrics = df_final_lyrics.drop(columns =['danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo'])
df_final_lyrics.to_csv("lyrics.csv")